In [2]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [17]:
import torch
from diffusers import FluxPipeline
from nunchaku.models.transformer_flux import NunchakuFluxTransformer2dModel
import time

# Load the model and pipeline
transformer = NunchakuFluxTransformer2dModel.from_pretrained("mit-han-lab/svdq-int4-flux.1-dev")
pipeline = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev", transformer=transformer, torch_dtype=torch.bfloat16
).to("cuda")

[2025-01-21 02:44:26.274] [info] Initializing QuantizedFluxModel
[2025-01-21 02:44:26.281] [info] Loading weights from /home/ubuntu/.cache/huggingface/hub/models--mit-han-lab--svdq-int4-flux.1-dev/snapshots/1357abfc3979e1036f1e246f2398f814de245d20/transformer_blocks.safetensors
[2025-01-21 02:44:27.573] [info] Done.


Expected types for transformer: ['FluxTransformer2DModel'], got NunchakuFluxTransformer2dModel.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.12it/s]


In [12]:
pip install para-attn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.5 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for para-attn: filename=para_attn-0.3.10-py2.py3-none-any.whl size=31162 sha256=ad5d5830fb0b27dd2bcde7ca61c4c1d92639cc70c0a7e4638e33c7cbdc8c74de
  Stored in directory: /home/ubuntu/.cache/pip/wheels/69/59/83/6faaa9fb85f80626b5cf622317180b76769b99d7942c682ef5
Successfully built para-attn
Note: you may need to restart the kernel to use updated packages.


In [23]:
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))  # Bind to an available port
        return s.getsockname()[1]

free_port = find_free_port()
print(f"Available port: {free_port}")


Available port: 43595


In [24]:
import os
import torch.distributed as dist

# Set required environment variables
os.environ["MASTER_ADDR"] = "localhost"  # Replace with the master node's IP if distributed
os.environ["MASTER_PORT"] = "43595"  # Replace with an available port
os.environ["WORLD_SIZE"] = "1"  # Total number of processes (1 for local testing)
os.environ["RANK"] = "0"  # Rank of the current process (0 for the master process)

# Initialize the process group
dist.init_process_group(
    backend="nccl",  # Use "gloo" for CPU
    init_method="env://",
    world_size=1,
    rank=0,
)

print("Process group initialized successfully!")


Process group initialized successfully!


In [25]:
import torch
import torch.distributed as dist
from diffusers import AutoencoderKL

# dist.init_process_group()

torch.cuda.set_device(dist.get_rank())

# vae = AutoencoderKL.from_pretrained(
#     "black-forest-labs/FLUX.1-dev",
#     torch_dtype=torch.bfloat16,
# ).to("cuda")

from para_attn.parallel_vae.diffusers_adapters import parallelize_vae

parallelize_vae(pipeline.vae)

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [28]:

# Measure execution time
start_time = time.time()
image = pipeline("A cat holding a sign that says hello world", num_inference_steps=50, guidance_scale=0).images[0]
end_time = time.time()

# Save the image
image.save("example.png")

# Print runtime
print(f"Image generation took {end_time - start_time:.2f} seconds.")

100%|██████████| 50/50 [00:18<00:00,  2.66it/s]


Image generation took 19.09 seconds.


In [29]:
import torch

# Total memory currently allocated on the GPU by PyTorch (in bytes)
allocated_memory = torch.cuda.memory_allocated() / (1024 ** 2)  # Convert to MB
print(f"Allocated memory: {allocated_memory:.2f} MB")

# Total memory currently cached by PyTorch (in bytes)
cached_memory = torch.cuda.memory_reserved() / (1024 ** 2)  # Convert to MB
print(f"Cached memory: {cached_memory:.2f} MB")

# Total memory capacity of the GPU (in bytes)
total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 2)  # Convert to MB
print(f"Total GPU memory: {total_memory:.2f} MB")

Allocated memory: 19237.60 MB
Cached memory: 22992.00 MB
Total GPU memory: 40326.38 MB
